# Maximal Representative Subsampling

In [70]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
np.seterr(divide = 'ignore') 

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from scipy.stats import kstest

%run utils.py

path = Path(os.getcwd()).parent


## Attribute Comparison

Each row is labelled with its (survey) *source*.

- *Source*
- Geschlecht
- Geburtsjahr
- Hoechster Bildungsabschluss
- Erwerbstaetigkeit
- Soziale Unterstuetzung

### Preprocessing: Allensbach

In [71]:
allensbach_cols = ['S01', 'S02', 'S03_rec', 'S05', 'V18']
allensbach_names = ['Geschlecht', 'Geburtsjahr', 'Hoechster Bildungsabschluss', 'Erwerbstaetigkeit',
                    'Soziale Unterstuetzung']

allensbach = pd.read_csv(os.path.join(path, 'data/Allensbach/abs_data.csv'),
                         usecols = allensbach_cols,
                         delimiter=';', low_memory=False)

allensbach.replace(',', '.', regex=True, inplace=True)
allensbach.replace(' ', np.nan, regex=True, inplace=True)

allensbach = allensbach.fillna(allensbach.median()) # quick fix, for the time being.
allensbach.rename(columns= dict(zip(allensbach_cols, allensbach_names)), inplace=True)

allensbach.head()

,Geschlecht,Geburtsjahr,Erwerbstaetigkeit,Soziale Unterstuetzung,Hoechster Bildungsabschluss
0,2,73,2,1,2
1,2,24,1,2,3
2,1,51,1,2,3
3,1,57,1,2,5
4,1,64,2,3,2


### Preprocessing: GBS

In [72]:
gbs1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs1.csv'))
gbs2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs2.csv'))

mapping1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map1.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')
mapping2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map2.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')

# Rename 'GBS-CODE' column
key = 'GBS-CODE'
mapping1 = mapping1.rename(columns={'GBS Code': key})
mapping2 = mapping2.rename(columns={'GBS Code': key})
gbs1 = gbs1.rename(columns={'gbs_code': key})
gbs2 = gbs2.rename(columns={'GBS-Code': key})

# Merge GBS "Umfrage Wellen" and create dataframe
gbs = pd.merge(gbs1, gbs2, how='inner', on=key)
gbs = pd.merge(gbs, mapping1, how='left', on=key)
gbs = pd.merge(gbs, mapping2, how='left', on=key)

gbs_cols = ['am01', 'am02_01', 'am14', 'am17', 'os03']
allensbach_names = ['Geschlecht', 'Geburtsjahr', 'Hoechster Bildungsabschluss', 
                    'Erwerbstaetigkeit', 'Soziale Unterstuetzung',]

gbs = gbs[gbs_cols]
gbs.rename(columns= dict(zip(gbs_cols, allensbach_names)), inplace=True)


del mapping1, mapping2, gbs1, gbs2

# TODO: Combine matching rows instead of selecting every 2nd.
gbs = gbs.iloc[::2, :]
gbs.head(5)

,Geschlecht,Geburtsjahr,Hoechster Bildungsabschluss,Erwerbstaetigkeit,Soziale Unterstuetzung
0,männlich,1979-04-02,"Abitur, allgemeine oder fachgebundene Hochschu...",1.0,viel Anteilnahme und Interesse
2,weiblich,1945-10-29,Hochschulabschluss,1.0,weder viel noch wenig
4,männlich,1949-09-06,Hochschulabschluss,1.0,viel Anteilnahme und Interesse
6,weiblich,1992-07-31,Hochschulabschluss,1.0,viel Anteilnahme und Interesse
8,weiblich,1984-03-01,Hauptschulabschluss/Volksschulabschluss,1.0,viel Anteilnahme und Interesse


In [ ]:
di1 = {'männlich': 0, "Männlich": 0, 'weiblich': 1, "Weiblich": 1, '[NA] Keine Angabe':np.nan}
df = df.replace({"Geschlecht": di1})

df['Geburtsjahr'] = df['Geburtsjahr'].astype(str).str[0:4].astype(int)

di3 = {'Türkei':0, 'Österreich': 0, 
       'Item nonresponse': np.nan, 'sonstige, und zwar:': 0, 'Europa':0, 'Deutschland':1, 'Andere':0}
df = df.replace({'Geburtsland': di3})

di4 = {2.0: 0, 2:0}
df = df.replace({'Nationalitaet': di4})

di5 = {'Verheiratet/ Eing. LP zus. lebend':1,
       'Verheiratet und lebe mit meinem/r Ehepartner/-in zusammen': 1, 
       'Geschieden/ Eing. LP aufgehoben': 2,
       'Geschieden': 2,
       'Ledig': 3,
       'In eingetragener Lebenspartnerschaft zusammenlebend (gleichgeschlechtlich)':1,
       'Verheiratet und lebe von meinem/meiner Ehepartner/-in getrennt': 0,
       'Eingetragene Lebenspartnerschaft, getrennt lebend (gleichgeschlechtlich)': 0} 

### Preprocessing: GESIS

We might need this later.

In [7]:
gesis1 = pd.read_csv(os.path.join(path, 'data/GESIS/ZA5665_a1_a11-a12_v22-0-0.dta.csv'), 
                     engine='python', encoding = "ISO-8859-1")
gesis2 = pd.read_csv(os.path.join(path, 'data/GESIS/ZA5665_a1_ca-cf_v22-0-0.dta.csv'), 
                     engine='python', encoding = "ISO-8859-1")
gesis3 = pd.read_csv(os.path.join(path, 'data/GESIS/ZA5665_a1_ba-bf_v22-0-0.dta.csv'), 
                     engine='python', encoding = "ISO-8859-1")
gesis4 = pd.read_csv(os.path.join(path, 'data/GESIS/ZA5665_a1_aa-ac_v22-0-0 (1).dta.csv'), 
                     engine='python', encoding = "ISO-8859-1")

gesis5 = pd.merge(gesis1, gesis2, how='inner', on='z000001a')
gesis6 = pd.merge(gesis3, gesis4, how='inner', on='z000001a')
gesis  = pd.merge(gesis5, gesis6, how='inner', on='z000001a')

del gesis1,gesis2,gesis3,gesis4,gesis5,gesis6

gesis = gesis[['a11d054a', 'a11d056b', 'a11d082b', 'a11d089c', 
              'a11d092a', 'acae051a', 'acae058a', 'acae062a', 
              'acae079a', 'acae089a', 'acae099a', 'acae085a', 
              'acae095a', 'acae105a', 'bdao099a']]
gesis.head(5)

,a11d054a,a11d056b,a11d082b,a11d089c,a11d092a,acae051a,acae058a,acae062a,acae079a,acae089a,acae099a,acae085a,acae095a,acae105a,bdao099a
0,Männlich,1946,"Fachhochschulreife, Fachoberschule",Nicht erwerbstätig,Missing by filter,Stimme zu,Weder noch,Missing by design,8,Missing by design,Missing by design,6,Missing by design,Missing by design,Mittelschicht
1,Weiblich,1974,Hauptschulabschluss,Vollzeiterwerbstätig,Angestellte(r),Weder noch,Stimme zu,Missing by design,Missing by design,5,Missing by design,Missing by design,3,Missing by design,Weiß ich nicht
2,Weiblich,1994,Schüler/-in,Nicht erwerbstätig,Missing by filter,Stimme zu,Lehne stark ab,Missing by design,9,Missing by design,Missing by design,6,Missing by design,Missing by design,Unit nonresponse
3,Männlich,1950,"Abitur, allgemeine oder fachgebundene Hochschu...",Vollzeiterwerbstätig,Angestellte(r),Stimme zu,Stimme stark zu,Missing by design,Missing by design,Missing by design,Item nonresponse,Missing by design,Missing by design,6,Keiner dieser Schichten
4,Männlich,1990,Realschulabschluss,Vollzeiterwerbstätig,Angestellte(r),Stimme zu,Missing by design,Lehne ab,8,Missing by design,Missing by design,5,Missing by design,Missing by design,Not reached


- "['a12d021a' 'a12d024a' 'dezg083a'] not in index"

## Experiment

In [ ]:
# Randomize data
df_low = df_low.reindex(np.random.permutation(df_low.index))
df_high = df_high.reindex(np.random.permutation(df_high.index))

rep = df.copy(deep=True)
non_rep = pd.concat([df_low.head(21000).copy(deep=True),
                     df_high.copy(deep=True)], sort=True)

print(rep['Above/Below 50K'].value_counts())
print(non_rep['Above/Below 50K'].value_counts())

rep['label'] = 'rep'
non_rep['label'] = 'nonrep'

In [ ]:
data = pd.concat([rep, non_rep], sort=True).copy(deep=True)
data.reset_index(drop=True, inplace=True)
data['probs'] = len(data.label)*[0]

temperature = 1 
max_drop = 1000
limit = 2000

ks = []
auc = []
ratio = []
acc = []

while (len(data[data.label == 'nonrep']) > max_drop and
       len(data.label) > limit):

    rf = RandomForestClassifier(n_estimators=100, 
                                bootstrap=True,
                                max_depth=5,
                                oob_score=True)
    
    probs = cross_val_predict(rf,
                              data.drop(['label', 'Above/Below 50K', 'probs'], axis=1),
                              data['label'], 
                              cv=3,
                              method='predict_proba')
    
    preds = cross_val_predict(rf,
                              data.drop(['label', 'Above/Below 50K', 'probs'], axis=1),
                              data['label'], 
                              cv=3)
    
    if preds[0] == 'nonrep' and round(probs[0][0], 0) == 1:
        data['probs'] = [p[0] for p in probs]
    else:
        data['probs'] = [p[1] for p in probs]
    
    drop_id = []
    for _ in range(max_drop):
        softmax = sample(data[data.label == 'nonrep']['probs'], temperature)
        drop = data[data.label == 'nonrep'].iloc[[np.argmax(softmax)]].index[0]
        drop_id.append(drop)
        
    data.drop(data.index[drop_id], inplace=True)
    data.reset_index(drop=True, inplace=True)

    
    # EVALUATION
    ratio.append(data[data.label == 'nonrep']['Above/Below 50K'].value_counts()[0]/
                 data[data.label == 'nonrep']['Above/Below 50K'].value_counts()[1])
    auc.append(roc([1 if k == 'nonrep' else 0 for k in data.label], data.probs, 1, 'name'))
    ks.append(kstest(data.probs, 'uniform'))
    
    print('length of current dataframe:', len(data[data.label == 'nonrep'].nonrep))